In [ ]:
import pyvisa

## 1) Connect to the waveform generator using pyvisa and the LAN interface

In [ ]:
def connect(ip_address = '192.168.2.92'):
    """
    Connects to the function generator through the ethernet interface 
    and confirms the connection by printing the instrument ID 

    :param ip_address: str, IP address of the function generator

    :return inst: pyvisa resource
    """
    rm = pyvisa.ResourceManager()
    inst = rm.open_resource(f'TCPIP0::{ip_address}::INSTR') 
    print(inst.query('*IDN?'))
    return inst

inst = connect()

## 2) Set basic waveform parameters

Refer to the SDG 2082X Programming manual to complete the following functions. For each function, you should complete the docstring where
it says "COMPLETE THIS"

Write a function to query the basic wave parameters and return a dictionary

In [ ]:
def get_basic_wave(inst, channel):
    """ Query the basic waveform parameters and return a dictionary that splits the parameters.

    :param inst: Pyvisa resource.
    :param channel: Waveform generator channel to query.
    """
    query = f'C{channel}:BSWV?'
    bswv = inst.query(query)
    params = bswv.split(' ')[1]
    params_split = params.split(',')
    bswv_params_dict = {
        param: val
        for param, val in zip(params_split[0::2], params_split[1::2])
    }
    
    return bswv_params_dict

In [ ]:
def get_channel_state(inst, channel):
    """ Get the current state of a channel.

    :param inst: Pyvisa resource.
    :param channel: Waveform generator channel.
    """
    query = f'C{channel}:OUTP?'
    state_str = inst.query(query)
    params = state_str.split(' ')
    state = params.split(',')[0]

    return state

Set the basic wave parameters

In [ ]:
def set_wave_type(inst, channel, wave_type):
    """ Set the waveform type using the Basic Wave command.

    :param inst: PyVisa resource object corresponding to the waveform generator.
    :param channel: (int) Channel on which to change the waveform type.
    :param wave_type: (str) one of ['SINE', 'SQUARE', 'RAMP', 'PULSE', 'ARB', 'DC', 'PRBS', 'IQ']
    """
    valid_wave_types = ['SINE', 'SQUARE', 'RAMP', 'PULSE', 'NOISE', 'ARB', 'DC', 'PRBS', 'IQ']
    if wave_type not in valid_wave_types:
        raise ValueError(f'wave_type must be one of {valid_wave_types}')
    cmd = f'C{channel}:BSWV WVTP,{wave_type}'
    inst.write(cmd)

def set_frequency(inst, channel, freq):
    """ Set the waveform fundamental frequency.
    
    :param inst: PyVisa resource object corresponding to the waveform generator.
    :param channel: (int) Channel on which to change the waveform type.
    :param freq: COMPLETE THIS
    """
    freq = float(abs(freq))
    freq_min = 1e-6
    freq_max = 1e6
    if not ((freq_min <= freq) and (freq <= freq_max)):
        raise ValueError(f'Specified frequency {freq} falls outside the range {freq_min} <= freq <= {freq_max}')
    cmd = f'C{channel}:BSWV FRQ,{freq}'
    inst.write(cmd)

def set_amplitude(inst, channel, amp):
    """ Set the waveform amplitude in peak-to-peak units.

    :param inst: PyVisa resource object corresponding to the waveform generator.
    :param channel: (int) Channel on which to change the waveform type.
    :param amp: COMPLETE THIS
    """
    amp = float(abs(amp))
    amp_min = 2e-3
    amp_max = 20
    if not ((amp_min <= amp) and (amp <= amp_max)):
        raise ValueError(f'Specified amplitude {amp} falls outside the range {amp_min} <= freq <= {amp_max}')
    cmd = f'C{channel}:BSWV AMP,{amp}'
    inst.write(cmd)

def set_offset(inst, channel, offset):
    """ Set the waveform DC offset.

    :param inst: PyVisa resource object corresponding to the waveform generator.
    :param channel: (int) Channel on which to change the waveform type.
    :param offset: COMPLETE THIS
    """
    offset = float(offset)
    
    # - first get the amplitude of the waveform - #
    amp = float(get_basic_wave(inst, channel)['AMP'][:-1])
    
    # - the max value of amp/2 + abs(offset) is 10 - #
    offset_check = (amp / 2) + abs(offset)
    max_offset = 10 - (amp / 2)
    if offset_check > 10:
        raise ValueError(f'Specified offset {offset}V exceeds the maximum value {max_offset} for the amplitude {amp}')

    # - set the offset - #
    cmd = f'C{channel}:BSWV OFST,{offset}'
    inst.write(cmd)

Turn a channel output on or off

In [ ]:
def set_channel_state(inst, channel, state):
    """ Set the state of a waveform generator channel to ON or OFF.

    :param inst: Pyvisa resource.
    :param channel: Waveform channel.
    :param state: 'ON' or 'OFF' to specify the state.
    """
    if state not in ['ON', 'OFF']:
        raise ValueError(f'Invalid state {state} specified! Must be "ON" or "OFF"!')
    cmd = f'C{channel}:OUTP {state}'
    inst.write(cmd)

## 3) Initialization

Initialize both channels of the waveform generator to a sine wave with frequency 1 Hz, amplitude 1 Vpp, offset 0V.

Make sure both channels are turned off

In [ ]:
def initialize(inst):
    """ Initialize both channels of the waveform generator to a sine wave with frequency 1 Hz, amplitude 1 Vpp, and offset 0V.
    Turn off both channels

    :param inst: Pyvisa resource.
    """
    for channel in [1, 2]:
        set_channel_state(inst, channel, 'OFF')
        set_wave_type(inst, channel, 'SINE')
        set_frequency(inst, channel, 1)
        set_amplitude(inst, channel, 1)
        set_offset(inst, channel, 0)

## 4) Logging

Use the get_basic_wave() function to create a log string of waveform parameters

In [ ]:
def create_log(channel, wave_type, freq, amp, offset):
    """ Create a string to log basic parameters of the SDG2082X waveform generator output.

    :param channel: Which channel the parameters correspond to.
    :param wave_type: Type of waveform.
    :param freq: Frequency in Hz.
    :param amp: Amplitude in volts peak-to-peak.
    :param offset: Offset voltage.
    """
    log = "# ------------- SDG2082X Waveform Settings ------------- #\n"
    log += f"Channel: {channel}\n"
    log += f"Waveform Type: {wave_type}\n"
    log += f"Frequency: {freq} Hz\n"
    log += f"Amplitude: {amp} Volts peak-to-peak\n"
    log += f"Offset: {offset} Volts\n"
    return log

In [ ]:
channel = 1
bswv = get_basic_wave(inst, channel)
log_str = create_log(
    channel=channel,
    wave_type=bswv['WVTP'],
    freq=float(bswv['FRQ'].split('HZ')[0]),
    amp=float(bswv['AMP'].split('V')[0]),
    offset=float(bswv['OFST'].split('V')[0])
)

In [ ]:
print(log_str)